In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
### In this script, we will take care of the precision queries.

In [2]:

import sys
sys.path.append('/nethome/jbang36/eko')


In [3]:
### let's run the experiment and get the results
video_names = ['cherry960', 'ua_detra960', 'dashcam960', 'jackson']
video_name = 'cherry960'
category = 'bus'

from eko_paper2022.motivation.main import *

images = load_dataset(video_name)


11-15-2022 11:31:50 [info:049]INFO : meta data of the video /srv/data/jbang36/video_data/cherry960/video.mp4 is (100000, 540, 960, 3)
meta data of the video /srv/data/jbang36/video_data/cherry960/video.mp4 is (100000, 540, 960, 3)
100%|██████████| 100000/100000 [01:52<00:00, 885.88it/s]


In [ ]:
images.shape

In [ ]:
category

In [ ]:

from eko_paper2022.motivation.main import execute_ekoalt
from eko_paper2022.experiments.main import *

nb_buckets = int(len(images) * 0.1)
eko = execute_ekoalt(images, video_name, category = category, nb_buckets = nb_buckets)
query, times = query_process_aggregate(eko)

a = query.y_pred
b = query.y_true
gt_aggregate = []
for bb in b:
    gt_aggregate.append(float(bb))
gt_aggregate = np.array(gt_aggregate)

### Real final version -- sigmoid function


In [ ]:
#### okay, finally, we move onto getting the sigmoid function working.....
#### to get this to work, basically we have to rewrite the propagate function
#### okay, function has been implemented, let's run it and see!!!!









### Okay, we need to redo this... let's just figure things out one by one

In [ ]:
### let's start by why this doesn't make any sense....okay the values that we get just by getting the iframes is as follows:

### things we must do.... first, let's change the video name, images, anchor_counts
### so for the labels, we need a really short version to test

### load tasti labels and save up to 100

import pandas as pd
directory = '/srv/data/jbang36/video_data/cherry960/tasti_labels.csv'
tmp = pd.read_csv(directory)




tmp[:100]

### save video of 100 cherry frames

from eko_utils.video_utils import save_np_to_compressed_video
images_short = images[:100]
np_arr = images_short
save_directory = '/srv/data/jbang36/video_data/cherry100/video.mp4'
save_np_to_compressed_video(np_arr, save_directory)



#### save this into the dataset

#### what about cherry short?
from eko_paper2022.motivation.main import *

video_name = 'cherry100'
images = load_dataset(video_name)



images.shape

cherry100_labels = tmp[:100]
cherry100_labels.to_csv('/srv/data/jbang36/video_data/cherry100/tasti_labels.csv')


In [ ]:
images.shape

In [7]:
anchor_count = int(len(images) * 0.1)

In [8]:
anchor_count

10000

In [9]:
### #### load the dataset and labels try eko on it -- both the original version and the new version

from eko_paper2022.experiments.main import execute_ekomab


eko_mab = execute_ekomab(images, video_name, category = category, nb_buckets = anchor_count)







index initializing....
number of training instances:  1
mining complete!
training complete!
inferring complete!
returning 8334 frames in 0.3926246203482151 seconds
bucketing complete!


Target DNN Invocations:   0%|          | 0/8335 [00:00<?, ?it/s]

In [ ]:
query, times = query_process_aggregate(eko_mab)


In [ ]:
### grab the final results
ekomab_aggregate = query.y_pred
ekomab_aggregate[2000:3000]

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize = (20,3))
start, end = 2850, 3000
plt.plot(gt_aggregate[start:end], label = 'gt')
plt.plot(ekomab_aggregate[start:end], label = 'eko')
plt.legend()

In [13]:

dnn_invocation = int(len(images) * 0.1)
from eko_paper2022.experiments.main import *


precision, recall = query_process_precision(eko_mab, dnn_invocation = dnn_invocation)
precision, recall

Sigmoid based Propagation:   0%|          | 0/100000 [00:00<?, ?it/s]

(6.227436823104693e-01, 0.80859375)

In [14]:
dnn_invocation = 20000

In [15]:
precision, recall = query_process_recall(eko_mab, dnn_invocation = dnn_invocation)
precision, recall

Sigmoid based Propagation:   0%|          | 0/100000 [00:00<?, ?it/s]

(2.56e-02, 1.0)

In [ ]:
#### now we go into individual methods and compute their numbers...
#### we will do precision and recall in 1 pass




#### SVM

In [ ]:

from eko_paper2022.experiments.svm_wrapper import SVMWrapper


svm_wrapper = SVMWrapper()
labels = gt_aggregate

st = time.perf_counter()
svm_wrapper.train(images, labels)
et  = time.perf_counter()

et - st


In [ ]:
### svm_precision

svm_retrieval = svm_wrapper.inference_retrieval(images)

In [ ]:
dnn_invocation = int(len(images) * 0.1)


In [ ]:

precision, recall = query_process_precision(None, dnn_invocation = dnn_invocation, y = (svm_retrieval, gt_retrieval))
precision, recall

In [ ]:
precision, recall = query_process_recall(None, dnn_invocation = dnn_invocation, y = (svm_retrieval, gt_retrieval))
precision, recall

## Resnet-18

In [ ]:
## Resnet-18

images.shape
### load the labels -- it's loading the csv file 
label_file = f'/srv/data/jbang36/video_data/{video_name}/tasti_labels.csv'

from eko_paper2022.experiments.resnet_wrapper import ResnetWrapper


resnet = ResnetWrapper()

labels = resnet._load_labels(images, label_file)



In [ ]:
### okay, let's try the training step..
st = time.perf_counter()
resnet.train(images, labels)
et = time.perf_counter()

In [ ]:
et - st

In [ ]:
resnet_retrieval = resnet.inference_retrieval(images, image_size = 224)

In [ ]:

precision, recall = query_process_precision(None, dnn_invocation = dnn_invocation, y = (resnet_retrieval, gt_retrieval))
precision, recall


In [ ]:
precision, recall = query_process_recall(None, dnn_invocation = dnn_invocation, y = (resnet_retrieval, gt_retrieval))
precision, recall


### EKO

In [ ]:
#### let's try eko

anchor_count = int( len(images) * 0.1 )

eko = execute_ekoalt(images, video_name, nb_buckets = anchor_count)


In [ ]:
### Q2:
dnn_invocation = int(len(images) * 0.1)

precision, recall = query_process_precision(eko, dnn_invocation = dnn_invocation)
precision, recall



In [ ]:
### Q2:
precision, recall = query_process_recall(eko, dnn_invocation = dnn_invocation)
precision, recall



### EKO I-frame only

In [ ]:

anchor_count = int( len(images) * 0.1 )

anchor_count

In [ ]:
#### let's try eko
from eko_paper2022.experiments.main import execute_ekomab

eko = execute_ekomab(images, video_name, nb_buckets = anchor_count)

In [ ]:
### Q2:
dnn_invocation = int(len(images) * 0.1)

precision, recall = query_process_precision(eko, dnn_invocation = dnn_invocation)
precision, recall


In [ ]:
### Q2:
precision, recall = query_process_recall(eko, dnn_invocation = dnn_invocation)
precision, recall

In [ ]:
reps = eko.reps

In [ ]:
len(reps)

In [ ]:
images.shape

### EKO I-frames + MAB

In [ ]:

anchor_count = int( len(images) * 0.1 )

anchor_count

In [ ]:
#### let's try a very vvery simple dataset

anchor_count = 20
video_length = 100
images_short = images[:100]

initial_anchors = [0, 10, 20, 30, 40, 50, 60, 70, 99]


In [ ]:
images_short.shape

In [ ]:
#### let's try eko
from eko_paper2022.experiments.main import execute_ekomab

eko = execute_ekomab(images, video_name, nb_buckets = anchor_count)

### Q2:
dnn_invocation = int(len(images_short) * 0.1)

precision, recall = query_process_precision(eko, dnn_invocation = dnn_invocation)
precision, recall

In [ ]:
reps = eko.reps

In [ ]:
len(reps), len(set(reps))

In [ ]:
eko.topk_reps.shape

In [ ]:
eko.topk_dists.shape

In [ ]:
eko.topk_dists

In [ ]:
### topk dists seems a bit wrong??

In [ ]:
len(reps)

In [ ]:
len(set(reps))

In [ ]:
### Q2:
precision, recall = query_process_recall(eko, dnn_invocation = dnn_invocation)
precision, recall

In [ ]:
len(reps)

In [ ]:
reps = eko.reps
min(reps), max(reps), len(reps)

### EKO I-frames + Sigmoid Function

In [ ]:
import matplotlib.pyplot as plt

x_axis = np.array([-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5])
y_axis = 1 / (1 + np.exp(-(x_axis-2)))

plt.plot(x_axis, y_axis)

### TASTIPT

In [ ]:
from eko_paper2022.motivation.main import execute_tastipt

tastipt = execute_tastipt(images, video_name, category = category, redo = False, image_size = 224, nb_buckets = anchor_count)



In [ ]:
### Q2:
### Q2:
precision, recall = query_process_precision(tastipt, dnn_invocation = dnn_invocation)
precision, recall



In [ ]:
### Q2:
precision, recall = query_process_recall(tastipt, dnn_invocation = dnn_invocation)
precision, recall



In [ ]:
def f1_score(precisions, recalls):
    f1s = []
    for precision, recall in zip(precisions, recalls):
        f1s.append(2* precision * recall / (precision + recall))
    return f1s

In [ ]:
#### Q2 additional records
names = ['eko_iframes', 'eko]

cherry_p = [0.96]
cherry_r = [0.96]

detrac_p = [0.98]
detrac_r = [0.97]

dashcam_p = [0.96]
dashcam_r = [0.95]

jackson_p = [0.78]
jackson_r = [0.71]

#### Q3 additional records
cherry_p = [0.96]
cherry_r = [0.95]

detrac_p = [0.98]
detrac_r = [0.95]

dashcam_p = [0.95]
dashcam_r = [0.94]

jackson_p = [0.66]
jackson_r = [0.79]


In [ ]:
#### Q2



x_axis_names = ['SVM', 'Resnet-18', 'TASTI-PT', 'EKO', 'EKO_mab', 'EKO_mab_sigmoid']


cherry_p = np.array([0.99, 0.97, 0.94, 0.96, 0.95, 0.95]) 
cherry_r = np.array([0.19, 0.52, 0.91, 0.96, 0.96, 0.96])
cherry_f = f1_score(cherry_p, cherry_r)

detrac_p = np.array([1.0, 0.96, 0.96, 0.98, 0.98, 0.98]) 
detrac_r = np.array([0.90, 0.94, 0.98, 0.97, 0.97, 0.97])
detrac_f = f1_score(detrac_p, detrac_r)

dashcam_p = np.array([1.0, 0.96, 0.94, 0.95, 0.96, 0.95]) 
dashcam_r = np.array([0.09, 0.61, 0.87, 0.92, 0.92, 0.92])
dashcam_f = f1_score(dashcam_p, dashcam_r)

jackson_p = np.array([1.0, 1.0, 0.63, 0.77, 0.62, 0.64]) 
jackson_r = np.array([0.26, 0.19, 0.8, 0.75, 0.70, 0.69])
jackson_f = f1_score(jackson_p, jackson_r)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib

font = {
        'weight' : 'bold',
        'size'   : 15}

matplotlib.rc('font', **font)
matplotlib.rc('ytick', labelsize=15)


x_axis_names = ['SVM', 'Resnet-18', 'TASTI-PT', 'EKO', 'EKO_mab', 'EKO_mab_sigmoid']

x = np.arange(len(x_axis_names))


### we need to do subplots
rows = 2
columns = 2
fig_size = (8, 5)
fig, axes = plt.subplots(rows, columns, sharey = False, sharex=True, figsize=fig_size)


i = 0
j = 0
width = 0.6

axes[i][j].bar(x_axis_names, cherry_f, width)
axes[i][j].set_ylabel('F1-Score', weight = 'bold', size = 15)
#axes[i][j].set_ylim(0, 200)
#axes[i][j].set_yticks([0, 50, 100, 150, 200])

axes[i][j].set_xticklabels(x_axis_names, rotation = 30, ha = 'right',rotation_mode='anchor')
#plt.ylim(0, 2)
#plt.xticks(x, x_axis_names)
axes[i][j].set_title('Cherry', weight = 'bold', size = 15)


i = 0
j = 1
axes[i][j].bar(x_axis_names, detrac_f, width)
axes[i][j].set_title('UA-DETRAC', weight = 'bold', size = 15)
axes[i][j].set_xticklabels(x_axis_names, rotation = 30, ha = 'right', rotation_mode='anchor')


i = 1
j = 0
axes[i][j].bar(x_axis_names, dashcam_f, width)
axes[i][j].set_xticklabels(x_axis_names, rotation = 30, ha = 'right', rotation_mode='anchor')
axes[i][j].set_ylabel('F1-Score', weight = 'bold', size = 15)

axes[i][j].set_title('Dashcam', weight = 'bold', size = 15)


i = 1
j = 1
axes[i][j].bar(x_axis_names, jackson_f, width)

axes[i][j].set_title('Jackson', weight = 'bold', size = 15)
axes[i][j].set_xticklabels(x_axis_names, rotation = 30, ha = 'right', rotation_mode='anchor')


fig.tight_layout()

#plt.legend()
plt.savefig('plots/query-execution-q2.png')
plt.show()

## Q3

In [ ]:
#### Q3



x_axis_names = ['SVM', 'Resnet-18', 'TASTI-PT', 'EKO', 'EKO_mab', 'EKO_mab_sigmoid']



cherry_p = np.array([0.63, 0.42, 0.91, 0.96, 0.95, 0.95]) 
cherry_r = np.array([0.96, 0.96, 0.92, 0.95, 0.95, 0.94])
cherry_f = f1_score(cherry_p, cherry_r)

detrac_p = np.array([0.94, 0.96, 0.98, 0.98, 0.98, 0.98]) 
detrac_r = np.array([0.96, 0.95, 0.95, 0.95, 0.95, 0.95])
detrac_f = f1_score(detrac_p, detrac_r)

dashcam_p = np.array([0.6, 0.7, 0.92, 0.95, 0.95, 0.95]) 
dashcam_r = np.array([0.96, 0.96, 0.91, 0.94, 0.94, 0.93])
dashcam_f = f1_score(dashcam_p, dashcam_r)

jackson_p = np.array([0.19, 0.04, 0.55, 0.70, 0.57, 0.60]) 
jackson_r = np.array([0.96, 0.98, 0.88, 0.85, 0.82, 0.79])
jackson_f = f1_score(jackson_p, jackson_r)

In [ ]:
cherry_f

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib

font = {
        'weight' : 'bold',
        'size'   : 15}

matplotlib.rc('font', **font)
matplotlib.rc('ytick', labelsize=15)


x_axis_names = ['SVM', 'Resnet-18', 'TASTI-PT', 'EKO', 'EKO_mab', 'EKO_mab_sigmoid']


x = np.arange(len(x_axis_names))


### we need to do subplots
rows = 2
columns = 2
fig_size = (8, 5)
fig, axes = plt.subplots(rows, columns, sharey = False, sharex=True, figsize=fig_size)


i = 0
j = 0
width = 0.6

axes[i][j].bar(x_axis_names, cherry_f, width)
axes[i][j].set_ylabel('F1-Score', weight = 'bold', size = 15)
#axes[i][j].set_ylim(0, 200)
#axes[i][j].set_yticks([0, 50, 100, 150, 200])

axes[i][j].set_xticklabels(x_axis_names, rotation = 30, ha = 'right',rotation_mode='anchor')
#plt.ylim(0, 2)
#plt.xticks(x, x_axis_names)
axes[i][j].set_title('Cherry', weight = 'bold', size = 15)


i = 0
j = 1
axes[i][j].bar(x_axis_names, detrac_f, width)
axes[i][j].set_title('UA-DETRAC', weight = 'bold', size = 15)
axes[i][j].set_xticklabels(x_axis_names, rotation = 30, ha = 'right', rotation_mode='anchor')


i = 1
j = 0
axes[i][j].bar(x_axis_names, dashcam_f, width)
axes[i][j].set_xticklabels(x_axis_names, rotation = 30, ha = 'right', rotation_mode='anchor')
axes[i][j].set_ylabel('F1-Score', weight = 'bold', size = 15)

axes[i][j].set_title('Dashcam', weight = 'bold', size = 15)


i = 1
j = 1
axes[i][j].bar(x_axis_names, jackson_f, width)

axes[i][j].set_title('Jackson', weight = 'bold', size = 15)
axes[i][j].set_xticklabels(x_axis_names, rotation = 30, ha = 'right', rotation_mode='anchor')


fig.tight_layout()

#plt.legend()
plt.savefig('plots/query-execution-q3.png')
plt.show()

In [ ]:
### Cherry Q1
cherry_q1 = np.array([3341, 2352, 1900, 1759]) * (1 / 140) + np.array([0, 0, 4.2, 4])
cherry_q2 = np.array([-1, -1, 300, 300]) * 1 / 140 + np.array([0, 0, 4.2, 4])
cherry_q3 = np.array([100, 100, 100, 600]) * 1 / 140 + np.array([0, 0, 4.2, 4])
detrac_q1 = np.array([55607, 28323, 23147, 20943]) * (1 / 140) + np.array([0, 0, 3.9, 4.5])
detrac_q2 = np.array([210, 210, 210, 210]) * (1 / 140) + np.array([0, 0, 3.9, 4.5])
detrac_q3 = np.array([100, 100, 110, 100]) * (1 / 140) + np.array([0, 0, 3.9, 4.5])

dashcam_q1 = np.array([20679, 14638, 8426, 7571]) * (1 / 140) + np.array([0, 0, 3.7, 3.9])
dashcam_q2 = np.array([-1, -1, 240, 210]) * (1 / 140) + np.array([0, 0, 3.7, 3.9])
dashcam_q3 = np.array([100, 100, 100, 100]) * (1 / 140) + np.array([0, 0, 3.7, 3.9])

dashcam_q1 = np.array([1316, 921, 1145, 1425]) * (1 / 140) + np.array([0, 0, 13, 12])
dashcam_q2 = np.array([-1, -1, -1, -1]) * (1 / 140) + np.array([0, 0, 13, 12])
dashcam_q3 = np.array([1500, 17600, -1, -1]) * (1 / 140) + np.array([0, 0, 13, 12])


In [ ]:
dashcam_q1

In [ ]:
THR = 1 / 140


#### Q1
x_axis_names = ['SVM', 'Resnet-18', 'TASTI-PT', 'EKO']
cherry_q1_others = [0, 0, 4.2, 3.9]
cherry_q1_post = [3300 * THR, 2545 * THR, 1892 * THR, 1829 * THR]

#### Q2
cherry_q2_others = [0, 0, 4.2, 3.9]
cherry_q2_post = [1219 * THR, 886 * THR, 779 * THR, 778 * THR]

#### Q3
cherry_q3_others = [0, 0, 4.2, 3.9]
cherry_q3_post = [1219 * THR, 886 * THR, 779 * THR, 778 * THR]


#### Q1
detrac_q1_others = [0, 0, 4.2, 3.9]
detrac_q1_post = [1219 * THR, 886 * THR, 779 * THR, 778 * THR]


#### Q2
detrac_q2_others = [0, 0, 4.2, 3.9]
detrac_q2_post = [1219 * THR, 886 * THR, 779 * THR, 778 * THR]


#### Q3
detrac_q3_others = [0, 0, 4.2, 3.9]
detrac_q3_post = [1219 * THR, 886 * THR, 779 * THR, 778 * THR]


#### Q1
dashcam_q1_others = [0, 0, 4.2, 3.9]
dashcam_q1_post = [1219 * THR, 886 * THR, 779 * THR, 778 * THR]


#### Q2
dashcam_q2_others = [0, 0, 4.2, 3.9]
dashcam_q2_post = [1219 * THR, 886 * THR, 779 * THR, 778 * THR]


#### Q3
dashcam_q3_others = [0, 0, 4.2, 3.9]
dashcam_q3_post = [1219 * THR, 886 * THR, 779 * THR, 778 * THR]


#### Q1
jackson_q1_others = [0, 0, 4.2, 3.9]
jackson_q1_post = [1219 * THR, 886 * THR, 779 * THR, 778 * THR]


#### Q2
jackson_q2_others = [0, 0, 4.2, 3.9]
jackson_q2_post = [1219 * THR, 886 * THR, 779 * THR, 778 * THR]


#### Q3
jackson_q3_others = [0, 0, 4.2, 3.9]
jackson_q3_post = [1219 * THR, 886 * THR, 779 * THR, 778 * THR]



import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


### we need to do subplots
rows = 1
columns = 3
i = 1
fig = plt.figure(figsize = (15,3))
fig.add_subplot(rows, columns, i)
plt.bar(x - width, cherry_values, width, label = 'Cherry')
plt.bar(x, ua_values, width, label = 'UA_DETRAC')
plt.bar(x + width, da_values, width, label = 'Dashcam')
plt.ylabel('Time (s)')
plt.xticks(x, x_axis_names)
plt.title('Q1 -- Aggregate')


i = 2
fig.add_subplot(rows, columns, i)
plt.bar(x - width, cherry_values, width, label = 'Cherry')
plt.bar(x, ua_values, width, label = 'UA_DETRAC')
plt.bar(x + width, da_values, width, label = 'Dashcam')
plt.xticks(x, x_axis_names)
plt.title('Q2 -- Precision')



i = 3
fig.add_subplot(rows, columns, i)
plt.bar(x - width, cherry_values, width, label = 'Cherry')
plt.bar(x, ua_values, width, label = 'UA_DETRAC')
plt.bar(x + width, da_values, width, label = 'Dashcam')
plt.xticks(x, x_axis_names)
plt.title('Q3 -- Recall')




plt.xticks(x, x_axis_names)
plt.legend()
plt.show()



In [ ]:
### TODO: Need to include the YOLOv5 inference time for EKO -- this is pretty significant

In [ ]:
### generate the outcome graph first 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
### drawing function for the graph

#### Q1 values

x_axis_names = ['YOLOv5', 'TASTI-PT', 'EKO']
cherry_values = [0, 9.68, 9.5]
ua_values = [0, 61.25, 65.4]
da_values = [0, 3.3 + 21.47, 2.9 + 22.17]


##### Q2 values
cherry_values = [0, 4.48 + 7.14, 4.26 + 7.14]
ua_values = [0, 11.18, 12.94]
da_values = [0, 3.57 + 7.14, 3.4 + 7.14]

##### Q3 values
cherry_values = [0, 4.56 + 7.14, 4.32 + 7.14]
ua_values = [0, 11.84, 11.04]
da_values = [0, 3.4 + 7.14, 3.4 + 7.14]


x = np.arange(len(x_axis_names))
width = 0.2


### we need to do subplots
rows = 1
columns = 3
i = 1
fig = plt.figure(figsize = (15,3))
fig.add_subplot(rows, columns, i)
plt.bar(x - width, cherry_values, width, label = 'Cherry')
plt.bar(x, ua_values, width, label = 'UA_DETRAC')
plt.bar(x + width, da_values, width, label = 'Dashcam')
plt.ylabel('Time (s)')
plt.xticks(x, x_axis_names)
plt.title('Q1 -- Aggregate')





i = 2
fig.add_subplot(rows, columns, i)
plt.bar(x - width, cherry_values, width, label = 'Cherry')
plt.bar(x, ua_values, width, label = 'UA_DETRAC')
plt.bar(x + width, da_values, width, label = 'Dashcam')
plt.xticks(x, x_axis_names)
plt.title('Q2 -- Precision')



i = 3
fig.add_subplot(rows, columns, i)
plt.bar(x - width, cherry_values, width, label = 'Cherry')
plt.bar(x, ua_values, width, label = 'UA_DETRAC')
plt.bar(x + width, da_values, width, label = 'Dashcam')
plt.xticks(x, x_axis_names)
plt.title('Q3 -- Recall')




plt.xticks(x, x_axis_names)
plt.legend()
plt.show()


In [ ]:
cherry_values = [0, 9.68, 9.5]
ua_values = [0, 61.25, 65.4]
da_values = [0, 3.3 + 21.47, 2.9 + 22.17]


##### Q2 values
cherry_values2 = [0, 4.48 + 7.14, 4.26 + 7.14]
ua_values2 = [0, 11.18, 12.94]
da_values2 = [0, 3.57 + 7.14, 3.4 + 7.14]

##### Q3 values
cherry_values3 = [0, 4.56 + 7.14, 4.32 + 7.14]
ua_values3 = [0, 11.84, 11.04]
da_values3 = [0, 3.4 + 7.14, 3.4 + 7.14]

In [ ]:
tmp = np.array([cherry_values, ua_values, da_values, cherry_values2, ua_values2, da_values2, cherry_values3, ua_values3, da_values3])
tmp.mean(axis = 0)

In [ ]:
print('hello world')